# Time Loss Experiments - Setup
Results are evaluated in the **032_Time-Loss-Experiments_Evaluation.ipynb**

In [1]:
import tensorflow as tf
from tensorflow import keras
import time

from functions_dataCreation import *
from functions_modelArchitectures import *
from class_LRFinder import *


from tensorflow.keras.callbacks import Callback

import pandas as pd

from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

set_x,set_y = readXYfromDisk(noOfFiles=10,folder="../data/Train") #Training data

set_y[set_y == 1] = 0
set_y[set_y == 2] = 1

IMG_SIZE = 128

METRICS = [
    #   keras.metrics.TruePositives(name='tp'),
    #   keras.metrics.FalsePositives(name='fp'),
    #   keras.metrics.TrueNegatives(name='tn'),
    #   keras.metrics.FalseNegatives(name='fn'), 
    #   keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 2060 SUPER, compute capability 7.5
X Shape :  (43614, 128, 128, 3) Memory size is : 2044.40625 Mb Y Shape:  (43614,)
Values, counts, Avg Performance :  [0 1 2] [23131  9960 10523] [0.53035722 0.22836704 0.24127574]


## Experiment I

The Base model has below parameters and then train three more model architecutures with a combination of Batch Normalisation and Dropout Layers

* Fixed :
    + No of CNN layers : 2
    + Filter Sizes : 10,20
    + Kernel size : 7x7
    + Pool size : Max Pooling 2x2
    + Dropout percentage : 0.1
    + FC Dense Layers : 2 x 128 units with relu
    + No of epochs : 20
    + Optimisation : RMSProp
    
    
* Variable
    + No if images : 10K and 40K
    + Learning Rates : 1e-2,1e-3,1e-4,1e-5 
    + Batch Sizes : 64,128,256    


In [1]:

modelBase = {}
modelBase['name'] = 'Base'
modelBase['inputShape'] = (IMG_SIZE,IMG_SIZE,3)
modelBase['activation'] = 'relu'

modelBase['filters'] = [10,20]
modelBase['convLayerMultiplier'] = 1
modelBase['kernelSize'] = (7,7)

modelBase['poolingLayer'] = 'MaxPooling2D'
modelBase['poolSize'] = (2,2)
modelBase['padding'] = 'same'

modelBase['denseLayers'] = 2
modelBase['units'] = 128
modelBase['activation'] = 'relu'

#with Dropout and BN
modelBase_with_BN_Dropout = modelBase.copy()
modelBase_with_BN_Dropout['name'] = 'modelBase_with_BN_Dropout'
modelBase_with_BN_Dropout['batchnormalization'] = True
modelBase_with_BN_Dropout['dropout'] = 0.1

#with Dropout only
modelBase_with_Dropout = modelBase.copy()
modelBase_with_Dropout['name'] = 'modelBase_with_Dropout'
modelBase_with_Dropout['dropout'] = 0.1

#with BN only
modelBase_with_BN = modelBase.copy()
modelBase_with_BN['name'] = 'modelBase_with_BN'
modelBase_with_BN['batchnormalization'] = True




X Shape :  (43614, 128, 128, 3) Memory size is : 2044.40625 Mb Y Shape:  (43614,)
Values, counts, Avg Performance :  [0 1 2] [23131  9960 10523] [0.53035722 0.22836704 0.24127574]


In [14]:
df = pd.DataFrame()

for mod in [modelBase, modelBase_with_BN_Dropout, modelBase_with_Dropout,modelBase_with_BN]:
        for images in [10000,40000]:
                for lr in [1e-2,1e-3,1e-4,1e-5]:
                        for bs in [64,128,256]:
                        
                                start_time = time.time()

                                set_x = set_x[:images]
                                set_y = set_y[:images]
                                
                                model = createCNN(mod)
                                
                                model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr)
                                        ,loss=tf.keras.losses.binary_crossentropy
                                        ,metrics=[METRICS])

                                history = model.fit(set_x,set_y
                                                ,batch_size = bs
                                                ,epochs=20
                                                ,verbose=0                                                
                                                )

                                temp = pd.DataFrame(history.history).rename_axis("epoch").reset_index()

                                print("Elapsed time ", round((time.time() - start_time)/60,2))
                                temp['elapsed'] = round((time.time() - start_time)/60,2)
                                temp['params'] = "Model_" + model.name + "_IMGs_" + str(images) + "_LR_" + str(lr) + "_BS_" + str(bs)

                                print("Model_" + model.name + "_IMGs_" + str(images) + "_LR_" + str(lr) + "_BS_" + str(bs))

                                df = df.append(temp)

df.to_pickle("./Time-Loss-Data_BNandDropouts.pkl")


Elapsed time  1.54
Model_modelBase_with_Dropout_IMGs_10000_LR_0.01_BS_64
Elapsed time  1.22
Model_modelBase_with_Dropout_IMGs_10000_LR_0.01_BS_128
Elapsed time  1.08
Model_modelBase_with_Dropout_IMGs_10000_LR_0.01_BS_256
Elapsed time  1.58
Model_modelBase_with_Dropout_IMGs_10000_LR_0.001_BS_64
Elapsed time  1.25
Model_modelBase_with_Dropout_IMGs_10000_LR_0.001_BS_128
Elapsed time  1.1
Model_modelBase_with_Dropout_IMGs_10000_LR_0.001_BS_256
Elapsed time  1.6
Model_modelBase_with_Dropout_IMGs_10000_LR_0.0001_BS_64
Elapsed time  1.25
Model_modelBase_with_Dropout_IMGs_10000_LR_0.0001_BS_128
Elapsed time  1.1
Model_modelBase_with_Dropout_IMGs_10000_LR_0.0001_BS_256
Elapsed time  1.6
Model_modelBase_with_Dropout_IMGs_10000_LR_1e-05_BS_64
Elapsed time  1.25
Model_modelBase_with_Dropout_IMGs_10000_LR_1e-05_BS_128
Elapsed time  1.13
Model_modelBase_with_Dropout_IMGs_10000_LR_1e-05_BS_256
Elapsed time  1.58
Model_modelBase_with_Dropout_IMGs_40000_LR_0.01_BS_64
Elapsed time  1.23
Model_modelBase_

,epoch,loss,precision,recall,auc,elapsed,params
0,0,2.384405,0.231844,0.033933,0.497361,1.54,Model_modelBase_with_Dropout_IMGs_10000_LR_0.0...
1,1,0.684890,0.000000,0.000000,0.500000,1.54,Model_modelBase_with_Dropout_IMGs_10000_LR_0.0...
2,2,0.684890,0.000000,0.000000,0.500000,1.54,Model_modelBase_with_Dropout_IMGs_10000_LR_0.0...
3,3,0.684890,0.000000,0.000000,0.500000,1.54,Model_modelBase_with_Dropout_IMGs_10000_LR_0.0...
4,4,0.684890,0.000000,0.000000,0.500000,1.54,Model_modelBase_with_Dropout_IMGs_10000_LR_0.0...
...,...,...,...,...,...,...,...
15,15,0.433874,0.640374,0.979150,0.981762,1.92,Model_modelBase_with_BN_IMGs_40000_LR_1e-05_BS...
16,16,0.422056,0.664367,0.984056,0.985370,1.92,Model_modelBase_with_BN_IMGs_40000_LR_1e-05_BS...
17,17,0.407500,0.683645,0.987735,0.990294,1.92,Model_modelBase_with_BN_IMGs_40000_LR_1e-05_BS...
18,18,0.392541,0.713069,0.992641,0.993168,1.92,Model_modelBase_with_BN_IMGs_40000_LR_1e-05_BS...


## Experiment II

The LR 1e-4, BS 128 model with 40K images was selected to evaluate the effect of different CNN Layer parameters

* Fixed :
    + No of Images : 40K
    + Batch Sizes : 128       
    + No of epochs : 40
    + Optimisation : RMSProp
    + Learning Rates : 1e-4
    + Dropout layers with percentage : 0.05
    + FC Dense Layers : 2 x 128 units with relu
    + Batch Normalisation
    
* Variable
    + No of layers : Deep, Deeper, Deepest
    + Filter Sizes : Variable
    + Kernel size : (3x3,7x7,15x15)
    + Pool size : Max Pooling (2x2,6x6,10x10)   
  



In [2]:
modelBase = {}
modelBase['name'] = 'Base'
modelBase['inputShape'] = (IMG_SIZE,IMG_SIZE,3)
modelBase['activation'] = 'relu'

modelBase['convLayerMultiplier'] = 1

modelBase['poolingLayer'] = 'MaxPooling2D'
modelBase['poolSize'] = (2,2)
modelBase['padding'] = 'same'

modelBase['denseLayers'] = 2
modelBase['units'] = 128
modelBase['activation'] = 'relu'

#with Dropout and BN
modelBase_with_BN_Dropout = modelBase.copy()
modelBase_with_BN_Dropout['name'] = 'modelBase_with_BN_Dropout'
modelBase_with_BN_Dropout['batchnormalization'] = True
modelBase_with_BN_Dropout['dropout'] = 0.05

filtersDict = {'deep' : [10,15,20,25,30]
                        ,'deeper':[10,15,20,25,30,35,40,45,50,55,60]
                        ,'deepest':[10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100]}


import pandas as pd
df = pd.DataFrame()

for fskey,fsvalue in filtersDict.items():
        for ks in [(3,3),(7,7),(15,15)]:
            for ps in [(2,2),(6,6),(10,10)]:

                start_time = time.time()

                set_x = set_x[:40000]
                set_y = set_y[:40000]
                
                modelBase_with_BN_Dropout['filters'] = fsvalue
                modelBase_with_BN_Dropout['kernelSize'] = ks
                modelBase_with_BN_Dropout['poolSize'] = ps

                model = createCNN(modelBase_with_BN_Dropout)
                
                model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4)
                        ,loss=tf.keras.losses.binary_crossentropy
                        ,metrics=[METRICS])

                history = model.fit(set_x,set_y
                                ,batch_size = 128
                                ,epochs=40
                                ,verbose=0                                                
                                )

                temp = pd.DataFrame(history.history).rename_axis("epoch")

                
                temp['elapsed'] = round((time.time() - start_time)/60,2)
                temp['params'] = fskey + "_Kernels_" + str(ks) + "_poolSize_" + str(ps)

                print("Elapsed time " + str(round((time.time() - start_time)/60,2)) + fskey + 
                                "_Kernels_" + str(ks) + "_poolSize_" + str(ps)) 

                df = df.append(temp)



df.to_pickle('./Time-Loss-Data_FilterKernelPool.pkl')


Elapsed time 10.83deep_Kernels_(3, 3)_poolSize_(2, 2)
Elapsed time 8.69deep_Kernels_(3, 3)_poolSize_(6, 6)
Elapsed time 8.53deep_Kernels_(3, 3)_poolSize_(10, 10)
Elapsed time 18.18deep_Kernels_(7, 7)_poolSize_(2, 2)
Elapsed time 13.61deep_Kernels_(7, 7)_poolSize_(6, 6)
Elapsed time 13.21deep_Kernels_(7, 7)_poolSize_(10, 10)
Elapsed time 60.01deep_Kernels_(15, 15)_poolSize_(2, 2)
Elapsed time 36.74deep_Kernels_(15, 15)_poolSize_(6, 6)
Elapsed time 35.35deep_Kernels_(15, 15)_poolSize_(10, 10)
Elapsed time 11.51deeper_Kernels_(3, 3)_poolSize_(2, 2)
Elapsed time 9.69deeper_Kernels_(3, 3)_poolSize_(6, 6)
Elapsed time 9.46deeper_Kernels_(3, 3)_poolSize_(10, 10)
Elapsed time 19.16deeper_Kernels_(7, 7)_poolSize_(2, 2)
Elapsed time 14.46deeper_Kernels_(7, 7)_poolSize_(6, 6)
Elapsed time 14.32deeper_Kernels_(7, 7)_poolSize_(10, 10)
Elapsed time 62.34deeper_Kernels_(15, 15)_poolSize_(2, 2)
Elapsed time 38.76deeper_Kernels_(15, 15)_poolSize_(6, 6)
Elapsed time 37.58deeper_Kernels_(15, 15)_poolSize


## Large kernel size means more training time, so lets keep it small and change depth and poolsize to see which ones gives low loss

* Depth of the network has no impact on the training time
* Pool Size also has not impact on training time

Create Deepest network
Keep Kernel size as 3,3
Try different pool sizes to get min loss

## Experiment III

The LR 1e-4, BS 128 model with 40K images was selected to evaluate the effect of different CNN Layer parameters

* Fixed :
    + No of Images : 40K
    + Batch Sizes : 128       
    + No of epochs : 60 (**Increased**)
    + Optimisation : RMSProp
    + Learning Rates : 1e-4
    + Dropout layers with percentage : 0.05
    + FC Dense Layers : 2 x 128 units with relu
    + Batch Normalisation
    + Kernel size : (3x3)
    
    
* Variable
    + No of layers : Deep, Deeper, Deepest
    + Filter Sizes : Variable    
    + Pool size : Max Pooling (2x2,6x6,10x10)   
  



In [2]:
modelBase = {}
modelBase['name'] = 'Base'
modelBase['inputShape'] = (IMG_SIZE,IMG_SIZE,3)
modelBase['activation'] = 'relu'

modelBase['convLayerMultiplier'] = 1

modelBase['poolingLayer'] = 'MaxPooling2D'
modelBase['poolSize'] = (2,2)
modelBase['padding'] = 'same'

modelBase['denseLayers'] = 2
modelBase['units'] = 128
modelBase['activation'] = 'relu'

#with Dropout and BN
modelBase_with_BN_Dropout = modelBase.copy()
modelBase_with_BN_Dropout['name'] = 'modelBase_with_BN_Dropout'
modelBase_with_BN_Dropout['batchnormalization'] = True
modelBase_with_BN_Dropout['dropout'] = 0.05

modelBase_with_BN_Dropout['kernelSize'] = (3,3)

filtersDict = {'deep' : [10,15,20,25,30]
                        ,'deeper':[10,15,20,25,30,35,40,45,50,55,60]
                        ,'deepest':[10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100]}


import pandas as pd
df = pd.DataFrame()

for fskey,fsvalue in filtersDict.items():        
        for ps in [(2,2),(6,6),(10,10)]:

            start_time = time.time()

            set_x = set_x[:40000]
            set_y = set_y[:40000]
            
            modelBase_with_BN_Dropout['filters'] = fsvalue                
            modelBase_with_BN_Dropout['poolSize'] = ps

            model = createCNN(modelBase_with_BN_Dropout)
            
            model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4)
                    ,loss=tf.keras.losses.binary_crossentropy
                    ,metrics=[METRICS])

            history = model.fit(set_x,set_y
                            ,batch_size = 128
                            ,epochs=60
                            ,verbose=0                                                
                            )

            temp = pd.DataFrame(history.history).rename_axis("epoch")

            
            temp['elapsed'] = round((time.time() - start_time)/60,2)
            temp['params'] = fskey + "_poolSize_" + str(ps)

            print("Elapsed time " + str(round((time.time() - start_time)/60,2)) + fskey + "_poolSize_" + str(ps)) 

            df = df.append(temp)
            df.to_pickle('./Time-Loss-Data_LayersPool.pkl')


Elapsed time 15.8deep_poolSize_(2, 2)
Elapsed time 13.12deep_poolSize_(6, 6)
Elapsed time 13.06deep_poolSize_(10, 10)
Elapsed time 17.29deeper_poolSize_(2, 2)
Elapsed time 14.75deeper_poolSize_(6, 6)
Elapsed time 14.31deeper_poolSize_(10, 10)
Elapsed time 19.8deepest_poolSize_(2, 2)
Elapsed time 16.41deepest_poolSize_(6, 6)
Elapsed time 16.46deepest_poolSize_(10, 10)


## Experiment IV

Same as above but stop training when loss reaches 0.45


In [2]:
class haltCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('loss') <= 0.45):
            print("\nReached 0.45 loss value so cancelling training!\n")
            self.model.stop_training = True



In [3]:
modelBase = {}
modelBase['name'] = 'Base'
modelBase['inputShape'] = (IMG_SIZE,IMG_SIZE,3)
modelBase['activation'] = 'relu'

modelBase['convLayerMultiplier'] = 1

modelBase['poolingLayer'] = 'MaxPooling2D'
modelBase['poolSize'] = (2,2)
modelBase['padding'] = 'same'

modelBase['denseLayers'] = 2
modelBase['units'] = 128
modelBase['activation'] = 'relu'

#with Dropout and BN
modelBase_with_BN_Dropout = modelBase.copy()
modelBase_with_BN_Dropout['name'] = 'modelBase_with_BN_Dropout'
modelBase_with_BN_Dropout['batchnormalization'] = True
modelBase_with_BN_Dropout['dropout'] = 0.05

modelBase_with_BN_Dropout['kernelSize'] = (3,3)

filtersDict = {'deep' : [10,15,20,25,30]
                        ,'deeper':[10,15,20,25,30,35,40,45,50,55,60]
                        ,'deepest':[10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100]}


import pandas as pd
df = pd.DataFrame()

for fskey,fsvalue in filtersDict.items():        
        for ps in [(2,2),(6,6),(10,10)]:

            start_time = time.time()

            set_x = set_x[:40000]
            set_y = set_y[:40000]
            
            modelBase_with_BN_Dropout['filters'] = fsvalue                
            modelBase_with_BN_Dropout['poolSize'] = ps

            model = createCNN(modelBase_with_BN_Dropout)
            
            model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4)
                    ,loss=tf.keras.losses.binary_crossentropy
                    ,metrics=[METRICS])

            history = model.fit(set_x,set_y
                            ,batch_size = 128
                            ,epochs=1000
                            ,verbose=0                   
                            ,callbacks=[haltCallback()]                             
                            )

            temp = pd.DataFrame(history.history).rename_axis("epoch")

            
            temp['elapsed'] = round((time.time() - start_time)/60,2)
            temp['params'] = fskey + "_poolSize_" + str(ps)

            print("Elapsed time " + str(round((time.time() - start_time)/60,2)) + fskey + "_poolSize_" + str(ps)) 

            df = df.append(temp)
            df.to_pickle('./Time-Loss-Data_LayersPool_Stop-training-at-loss0.5.pkl')





Reached 0.45 loss value so cancelling training!



Elapsed time 21.68deep_poolSize_(2, 2)
Elapsed time 215.77deep_poolSize_(6, 6)


KeyboardInterrupt: 

## Experiment V: NOT DONE
## Above models not learning fast enough

* Try the above setup with lower and higher learning rates. 
* Try 60 epochs unless loss drops to 0.45




## Experiment VI
 
### For the deeper network, keep everything fixed and get the best learning rate
* Also reduce dropout and remove BN ( as BN tends to take time)
* try with 10K and 40kimages


In [1]:
import tensorflow as tf
from tensorflow import keras
import time

from functions_dataCreation import *
from functions_modelArchitectures import *
from class_LRFinder import *


from tensorflow.keras.callbacks import Callback

import pandas as pd

import wandb
from wandb.keras import WandbCallback

wandb.init(project="candlestick-CNN", name = 'Deeper 1e-5 10K epochs' )


set_x,set_y = readXYfromDisk(noOfFiles=20,folder="../data/Train") #Training data
test_set_x,test_set_y = readXYfromDisk(noOfFiles=2,folder='../data/Test') 

set_y[set_y == 1] = 0
set_y[set_y == 2] = 1

test_set_y[test_set_y == 1] = 0
test_set_y[test_set_y == 2] = 1

IMG_SIZE = 128

METRICS = [
    #   keras.metrics.TruePositives(name='tp'),
    #   keras.metrics.FalsePositives(name='fp'),
    #   keras.metrics.TrueNegatives(name='tn'),
    #   keras.metrics.FalseNegatives(name='fn'), 
    #   keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]


Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: amitagni (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


X Shape :  (85064, 128, 128, 3) Memory size is : 3987.375 Mb Y Shape:  (85064,)
Values, counts, Avg Performance :  [0 1 2] [44005 20023 21036] [0.51731637 0.23538747 0.24729615]
X Shape :  (10060, 128, 128, 3) Memory size is : 471.5625 Mb Y Shape:  (10060,)
Values, counts, Avg Performance :  [0 1 2] [4988 2453 2619] [0.49582505 0.24383698 0.26033797]
